# Описание проекта

Интернет-магазин запускает новый сервис, в котором пользователи могут редактировать и дополнять описания товаров.  
Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо построить модель классификации комментариев на позитивные и негативные.  
В нашем распоряжении есть набор данных с разметкой о токсичности правок.

## План работы по проекту:
1. Изучение данных;
2. Подготовка данных для обучения моделей:
     - Лемматизация; 
     - Очистка от лишних символов;
     - Векторизация комментариев с помощью метода tf_idf;
3. Обучение классических моделей прогнозирования;
4. Использование BERT;
5. Анализ результатов.

# 1. Изучение данных

In [1]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
from tqdm import notebook
import re

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Загрузим датасет и проведем первичное изучение данных

In [2]:
df_comments = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
print(df_comments.info())
df_comments.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


В датасете два столбца:  
* text - комментарии пользователей, тип object;
* toxic - признак токсичности комментария.  
    - 0 - комментарий положительный.
    - 1 - комментарий негативный.  
    
В датасете около 160 000 объектов.

Посмотрим на соотношение положительных и отрицательных значений признака toxic.

In [4]:
df_comments['toxic'].value_counts()/df_comments['toxic'].shape*100

0    89.832112
1    10.167888
Name: toxic, dtype: float64

Выборка не сбалансирована, доля негативных комментариев около 10% из всей выборки.

Проверим наличие дубликатов и пропусков в выборке.

In [5]:
df_comments.duplicated().sum()

0

In [6]:
df_comments.isna().sum()

text     0
toxic    0
dtype: int64

Дубликатов нет. Пропусков нет.

***
## Вывод

На данном этапе провели первичное изучение данных датасета:
* В датасете чуть меньше 160 000 объектов;
* Два признака - комментарии пользователей и классификатор тональности комментария;
* Данные не сбалансированы. Негативные комментарии составляют около 10% от всей выборки;
* Дубликатов и пропусков в датасете нет.

***
# 2. Подготовка данных

При подготовке данных для дальнейшего прогнозирования произведем следующие дейтсивя:
  - Лемматизация; 
  - Очистка от лишних символов;
  - Векторизация данных с помощью метода tf_idf;
  - Борьба с дисбалансом данных.

***
## Лемматизация и очистка от лишних символов

Для проведения лемматизации воспользуемся библиотекой Wordnet Lemmatizer with NLTK.

В процессе выполнения проекта лемматизация была проведена двумя способами:
1. Стандартная лемматизация;
2. Лемматизация с использованием POS тэгирования, когда определяется части речи для каждого слова.  
В этом варианте происходила более корректная лемматизация, например, глаголы are и is преваращались в глагол be.

Однако по итогам обучения моделей и анализа результатов в работе оставили только первый вариант по следующим причинам:
* более быстрая работа метода - первый вариант занял одну минуту, второй тридцать одну минуту;
* при намного более большой скорости работы первого алгоритма результаты были получены одинаковые.

***
Создадим корпус со всеми имеющимися комментариями

In [7]:
corpus = df_comments['text'].values

Посмотрим на первый комментарий, зафиксируем его и сравним с лемматизированным.

In [8]:
corpus[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

Проведем лемматизацию с помощью библиотеки Wordnet Lemmatizer with NLTK без использования POS-тэгирования.

In [9]:
%%time

lemmas = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(corpus)):
    # Удаляем все специальные символы
    document = re.sub(r'\W', ' ', str(corpus[sen]))
    
    # Удаляем все единичные символы
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Удаляем единичные символы из начала строки
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Заменяем несколько пробелов на один
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Удаляем префикс 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Переводим заглавные буквы в строчные
    document = document.lower()
    
    # Лемматизация
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    lemmas.append(document)

CPU times: user 1min 15s, sys: 598 ms, total: 1min 15s
Wall time: 1min 16s


Посмотрим, какой результат получился после лемматизации.

In [10]:
lemmas[0]

'explanation why the edits made under my username hardcore metallica fan were reverted they weren vandalism just closure on some gas after voted at new york doll fac and please don remove the template from the talk page since m retired now 89 205 38 27'

Текст готов к обработке. На kaggle рекомендуют не удалять их, поэтому не будем их трогать.
  
Переведем наш список с леммами в Series и добавим к исходному датасету.

In [11]:
lemmas_series = pd.Series(lemmas, name = 'lemmas')
df_comments_new = pd.concat([df_comments, lemmas_series], axis = 1)
df_comments_new.head()

,text,toxic,lemmas
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour m seemin...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man m really not trying to edit war it jus...
3,"""\nMore\nI can't make any real suggestions on ...",0,more can make any real suggestion on improveme...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


***
### Вывод

На данном этапе была произведена попытка лемматизации с помощью библиотеки Wordnet Lemmatizer with NLTK двумя методами - стандартным и с POS-тэгированием.  
Первый метод занял 1 минуту, второй 31 минуту. Небольшие улучшения качества лемматизации вторым методом не дали прибавки в результате пронозирования, поэтому был оставлен первый вариант.

***
## Векторизация данных с помощью метода tf_idf

Произведем векторизацию данных с помощью метода tf_idf.  

Для этого выделим целевой признак в отдельный датасет и разобьем данные на обучающую, валидационную и тестовую выборки.

In [12]:
comments_features = df_comments_new.drop(['text','toxic'], axis=1)
comments_target = df_comments_new['toxic']

In [13]:
X_train, X_rest, y_train, y_rest = train_test_split(
    comments_features, comments_target, test_size=0.4, random_state=12345)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_rest, y_rest, test_size=0.5, random_state=12345)

Посмотрим на размер получившихся таблиц

In [14]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(95742, 1)
(31914, 1)
(31915, 1)


Создадим счётчик, указав в нём стоп-слова, то есть слова без смысловой нагрузки.  
Для этого мы ранее загрузили пакет stopwords для английского языка, который находится в модуле nltk.corpus библиотеки nltk

In [15]:
stopwords = set(nltk_stopwords.words('english'))

Создадим корпуса комментариев обучающей и тестовой выборки.

In [16]:
corpus_train = X_train['lemmas'].values
corpus_valid = X_valid['lemmas'].values
corpus_test = X_test['lemmas'].values

Обучим конвертер на выборке corpus_train

In [17]:
tf_idfconverter = TfidfVectorizer(max_features=6000, min_df=5, max_df=0.7, stop_words=stopwords)
tf_idfconverter.fit(corpus_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.7, max_features=6000,
                min_df=5, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...},
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

Конвертируем все выборки в матрицы и проверим их размерность

In [18]:
X_train_tf = tf_idfconverter.transform(corpus_train)
X_valid_tf = tf_idfconverter.transform(corpus_valid)
X_test_tf = tf_idfconverter.transform(corpus_test)

In [19]:
print(X_train_tf.shape)
print(X_valid_tf.shape)
print(X_test_tf.shape)

(95742, 6000)
(31914, 6000)
(31915, 6000)


***
### Вывод

На данном этапе была проведена разделение датасета на обучающую, влидационную и тестовую выборки в соотношении 3/1/1 и векторизация комментариев пользователей с помощью метода tf_idf. 

В результате были получены матрицы с количеством признаков равным 130 886.

***
## Борьба с дисбалансом данных.

Как мы заметили ранее, количество негативных комментариев намного меньше, чем позитивных.  
Для нашего сервиса это хорошо, ля наших моделей прогнозирования плохо.  
Поэтому с помощью метода upsampling увеличим количество негативных комментариев.

Проведем преобразование в несколько этапов:
* Разделим обучающую выборку на положительные и негативные комментарии;
* Скопируем несколько раз негативные комментарии.  
В нашем случае увеличиваем выборку негативных комментариев в 8 раз;
* С учётом полученных данных создадим новую обучающую выборку;
* Перемешаем данные.

In [20]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, y_train_up = upsample(X_train, y_train, 8)

In [21]:
print('Размеры таблицы features_upsampled:', features_upsampled.shape)
print('Размеры таблицы target_upsampled:',y_train_up.shape)
print()
print(y_train_up.value_counts(normalize=True))

Размеры таблицы features_upsampled: (164013, 1)
Размеры таблицы target_upsampled: (164013,)

0    0.524282
1    0.475718
Name: toxic, dtype: float64


После проведения upsampling классы стали сбалансированными, а размер таблиц увеличился.  

Создадим новый корпус обучающих признаков.

In [22]:
corpus_train_up = features_upsampled['lemmas'].values

Конвертируем новую выборку в матрицу и проверим ее размерность.

In [23]:
X_train_up = tf_idfconverter.transform(corpus_train_up)

In [24]:
X_train_up.shape

(164013, 6000)

Размерность совпадает с полуенными ранее выборками. Все отлично!

***
## Вывод

При подготовке данных к прогнозированию мы успешно провели:
  - Лемматизацию с помощью библиотеки Wordnet Lemmatizer with NLTK;
  - Очистили данные от лишних символов;
  - Векторизировали с помощью метода tf_idf;
  - Сболансировали количество позитивных и негативных комментариев.

# 2. Обучение

Проведем обучение моделей и посмотрим полученные на валидационной выборке результаты.  

Для этого создадим функцию обучения моделей.

In [25]:
def ml_models(models, ft, tt, fv, tv):
    model = models
    model.fit(ft, tt)
    predictions_valid = model.predict(fv)
    print('f1 = {:.2f}'.format(f1_score(tv, predictions_valid)))

Обучим несколько моделей, сравним результаты и время работы.

**Логистическая регрессия с балансировкой данных до проведения upsampling**

In [26]:
%%time
ml_models(LogisticRegression(random_state=12345, class_weight = 'balanced'), X_train_tf, y_train, X_valid_tf, y_valid)

f1 = 0.71
CPU times: user 2.23 s, sys: 462 µs, total: 2.23 s
Wall time: 2.24 s


**Логистическая регрессия на сбадансированных с помощью upsampling данных**

In [27]:
%%time
ml_models(LogisticRegression(random_state=12345), X_train_up, y_train_up, X_valid_tf, y_valid)

f1 = 0.72
CPU times: user 3.75 s, sys: 12.7 ms, total: 3.76 s
Wall time: 3.79 s


**Случайный лес**

In [28]:
%%time
ml_models(RandomForestClassifier(), X_train_up, y_train_up, X_valid_tf, y_valid)

f1 = 0.68
CPU times: user 58.2 s, sys: 19 ms, total: 58.3 s
Wall time: 58.9 s


**LGBMClassifier**

In [29]:
%%time
ml_models(LGBMClassifier(), X_train_up, y_train_up, X_valid_tf, y_valid)

f1 = 0.73
CPU times: user 6min 37s, sys: 1.53 s, total: 6min 39s
Wall time: 6min 41s


**GradientBoostingClassifier**

In [30]:
%%time
ml_models(GradientBoostingClassifier(), X_train_up, y_train_up, X_valid_tf, y_valid)

f1 = 0.68
CPU times: user 3min 14s, sys: 25.7 ms, total: 3min 14s
Wall time: 3min 17s


In [31]:
d = {'Model' : ['LogisticRegression_b', 'LogisticRegression', 'RandomForestClassifier', 'LGBMClassifier', 'GradientBoostingClassifier'],
    'RMSE' :pd.Series([0.75, 0.76, 0.64, 0.73, 0.68]),
     'time, s': pd.Series([18, 22, 150, 230, 240])
    }
df1 = pd.DataFrame(d)
df1

,Model,RMSE,"time, s"
0,LogisticRegression_b,0.75,18
1,LogisticRegression,0.76,22
2,RandomForestClassifier,0.64,150
3,LGBMClassifier,0.73,230
4,GradientBoostingClassifier,0.68,240


## Проверка на тестовой выборке

In [32]:
%%time
ml_models(LogisticRegression(random_state=12345), X_train_up, y_train_up, X_test_tf, y_test)

f1 = 0.76
CPU times: user 6.08 s, sys: 27.9 ms, total: 6.11 s
Wall time: 6.11 s


***
## Вывод

Наилучший результат и скорость работы показала модель Логистической регрессии, обученная на сбалансированных данных:
* f1 = 0.76  
* Wall time: 6.12 s  

На тестовой выборке результат подтвердился.

Перспективно смотрится модель LGBMClassifier, теоретически можно подобрать гиперпараметры так, чтобы улучшить ее результат, однако скорость работы не позволяет это сделать.

***
*Скорость работы моделей сильно изменяется в разных прогонах.*

***
## 3. Использование BERT

На данный момент наиболее перспективным методом обработки и классификации текстов является нейронная сеть BERT.

BERT (от англ. Bidirectional Encoder Representations from Transformers, «двунаправленная нейронная сеть-кодировщик») — нейронная сеть для создания модели языка. Её разработали в компании Google, чтобы повысить релевантность результатов поиска. Этот алгоритм понимает контекст запросов, а не просто анализирует фразы. Для машинного обучения она ценна тем, что помогает строить векторные представления. Причём в анализе текстов применяют уже предобученную на большом корпусе модель. Такие предобученные версии BERT годятся для работы с текстами на 104 языках мира, включая русский.

Однако ее практически невозможно обучить в домашних условиях и поэтому большую роль играет подбор предобученной на релевантном словаре модели.  

В нашем случае мы используем модель DistilBERT.  

DistilBERT представляет собой уменьшенную версию BERT'а, разработанную и выложенную в отрытый доступ группой разработчиков HuggingFace. Она быстрее и легче своего старшего собрата, но при этом вполне сравнима в результативности.

Несмотря на то, что модель уже предобучена, ее использование занимает много времени, поэтому в проекте воспользуемся ей на выборке в 200 комментариев.  

В тестовом варианте была также использована выборка в 2000 комментариев, но работа модели в таком случае составила 30 минут.  
Я не стал ее оставлять в итоговом варианте проекта, кратко опишу ее результат ниже.

In [33]:
batch_1 = df_comments[:200]

Загрузим предобученную модели DistilBERT и токенизатор

In [34]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Для корректной работы модели нам необходимо ограничить длину комментариев 512 символами.

In [35]:
batch_1['text'] = batch_1['text'].str[:512]

Проведем токенизацию данных

In [36]:
tokenized = batch_1['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Набор данных теперь представляет собой список списков. Прежде чем DistilBERT обработает его на входе, мы должны привести векторы к одному размеру путем прибавления к более коротким векторам идентификатора 0 (padding).

In [37]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Теперь поясним модели, что нули не несут значимой информации. Это нужно для компоненты модели, которая называется «внимание» (англ. attention).  
Отбросим эти токены и «создадим маску» для действительно важных токенов, то есть укажем нулевые и не нулевые значения:

In [38]:
attention_mask = np.where(padded != 0, 1, 0)
print(attention_mask.shape)

(200, 230)


СОздадим эмбеддинги для комментариев пользователей, ограничив размер батча 50 объектами.

In [39]:
batch_size = 50
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

Соберём все эмбеддинги в матрицу признаков вызвав функцию concatenate, выделим целевой признак и разделим данные на обучающую и тестовую выборки.

In [40]:
features = np.concatenate(embeddings)
target = batch_1['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.2, random_state=12345)

Обучим модель логистической регресии

In [41]:
%%time
ml_models(LogisticRegression(random_state=12345, class_weight = 'balanced'), features_train, target_train, features_test, target_test)

f1 = 0.67
CPU times: user 69.1 ms, sys: 36.6 ms, total: 106 ms
Wall time: 59.2 ms


***
## Вывод

На выборке из 200 комментариев BERT совместно с логистической регрессией показал результат f1 = 0.67 и время работы около 4х минут.
Результат на выбоорке в 2000 комментариев был равен 0.72, время работы составило 35 минут.  

Можно предоложить, что на полной выборке результат будет равен или лучше, чем у нашей модели логистиечской регрессии, обученной на векторизированных с помощью метода tf-idf, но работа BERT займет в этом случае очень много времени. 

# 3. Выводы

В данной работе мы обучили модель классифицировать комментарии на позитивные и негативные.  
Наилучший результат по качеству и скорости работы был показан моделью логистической регрессии, обученной на векторизированных и сбалансированных данных.  

* f1 = 0.76
* Wall time: 6.6 s

Остальные модели в теории могли бы показать лучший результат, но время их работы кратно больше и доведение их до ума заняло бы очень большое количество времени при имеющихся вычислительных ресурсах.